# Nettoyage des données en provenance d'Enedis

Ce notebook permet de nettoyer les données depuis Enedis. Le fichier est au format CSV mais les données ne sont pas facilement accessible. Les deux premières lignes sont des en-tête pour définir la période ainsi que différente définition tel que l'unité de mesure.

In [1]:
BASE_PATH = "datalake/"
RAW_DATA_FILENAME = "sample.csv"
RAW_DATA_CLEANED_FILENAME = "enedis_conso_clean.csv"
CLEANED_DATA_FILENAME = "enedis-data-ready-to-work.parquet"


In [2]:
with open(f"{BASE_PATH}{RAW_DATA_FILENAME}", "r+") as fp:
    lines = fp.readlines()
    with open(f"{BASE_PATH}{RAW_DATA_CLEANED_FILENAME}", "w+") as fpclean:
        fpclean.writelines(lines[2 : len(lines) - 2])


In [3]:
import pandas as pd

df = pd.read_csv(f"{BASE_PATH}{RAW_DATA_CLEANED_FILENAME}", sep=";")


In [4]:
df["Total Consommation"] = df["EAS F2"] + df["EAS F1"] + df["EAS F3"]


In [5]:
df_work = df[["Horodate", "EAS F1", "EAS F2", "EAS F3", "Total Consommation"]].copy()
df_work.columns = [
    "Horodate",
    "Quantité Heure Creuse en wh",
    "Quantité Heure Pleine en wh",
    "Quantité Weekend en wh",
    "Total Consommation",
]


In [6]:
df_work["Différence journalier Total Consommation"] = df_work[
    "Total Consommation"
].diff()
df_work["Différence journalier Heure Creuse"] = df_work[
    "Quantité Heure Creuse en wh"
].diff()
df_work["Différence journalier Heure Pleine"] = df_work[
    "Quantité Heure Pleine en wh"
].diff()
df_work["Différence journalier Weekend"] = df_work["Quantité Weekend en wh"].diff()
df_work


,Horodate,Quantité Heure Creuse en wh,Quantité Heure Pleine en wh,Quantité Weekend en wh,Total Consommation,Différence journalier Total Consommation,Différence journalier Heure Creuse,Différence journalier Heure Pleine,Différence journalier Weekend
0,2021-04-01T01:00:00+02:00,11538434,12944147,3668104,28150685,NaN,NaN,NaN,NaN
1,2021-04-02T01:00:00+02:00,11543332,12948970,3668104,28160406,9721.0,4898.0,4823.0,0.0
2,2021-04-03T01:00:00+02:00,11550420,12954810,3668104,28173334,12928.0,7088.0,5840.0,0.0
3,2021-04-04T01:00:00+02:00,11550420,12954810,3683924,28189154,15820.0,0.0,0.0,15820.0
4,2021-04-05T01:00:00+02:00,11550420,12954810,3700863,28206093,16939.0,0.0,0.0,16939.0
5,2021-04-06T01:00:00+02:00,11550420,12954810,3715235,28220465,14372.0,0.0,0.0,14372.0
6,2021-04-07T01:00:00+02:00,11558166,12962724,3715235,28236125,15660.0,7746.0,7914.0,0.0
7,2021-04-08T01:00:00+02:00,11567763,12973381,3715235,28256379,20254.0,9597.0,10657.0,0.0
8,2021-04-09T01:00:00+02:00,11576991,12983585,3715235,28275811,19432.0,9228.0,10204.0,0.0
9,2021-04-10T01:00:00+02:00,11586225,12989519,3715235,28290979,15168.0,9234.0,5934.0,0.0


In [7]:
df_work.to_parquet(f"{BASE_PATH}{CLEANED_DATA_FILENAME}")
